In [ ]:
!pwd

In [ ]:
# Clone the repository
!git clone https://github.com/huggingface/lerobot.git
%cd lerobot

In [ ]:
# lerobot source code is built with old versions of PyTorch, which doesn't
# support latest Nvidia GPU and CUDA. Update pyproject.toml to install latest
# PyTorch versions
import re
import os

def patch_pyproject(file_path="pyproject.toml"):
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found.")
        return

    with open(file_path, "r") as f:
        lines = f.readlines()

    # Define the exact new strings you want inside the first set of quotes
    updates = {
        "torch": "torch>=2.2.1",
        "torchcodec": "torchcodec>=0.2.1",
        "torchvision": "torchvision>=0.21.0"
    }

    new_lines = []
    for line in lines:
        patched_line = line
        for package, new_full_string in updates.items():
            # This regex looks for the package name inside quotes and replaces
            # EVERYTHING inside those specific quotes up to the first ';' or the end quote.
            # It handles multiple comma-separated constraints by wiping them out.
            pattern = rf'"{package}[^a-zA-Z].*'
            replacement = f'"{new_full_string}",'
            patched_line = re.sub(pattern, replacement, patched_line)
        new_lines.append(patched_line)

    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print("Cleanly patched pyproject.toml and removed legacy constraints.")

patch_pyproject()

!git diff

In [ ]:
# Install the library in editable mode with training dependencies
!apt update && apt install -y ffmpeg
!pip install "torch==2.10.0" \
    "torchvision>=0.21.0" \
    "torchcodec==0.10.0" \
    "transformers==4.57.6" \
    --extra-index-url https://download.pytorch.org/whl/cu130
!pip install -e .
!pip install wandb  # Highly recommended for monitoring your FLL progress
!pip install python-dotenv

In [ ]:
# Setting up env
import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)

def get_secret(key_name):
    # 1. Try Google Colab userdata (Secret Manager)
    if 'google.colab' in sys.modules:
        try:
            from google.colab import userdata
            return userdata.get(key_name)
        except Exception:
            pass # Fallback if secret not found in Colab
            
    # 2. Try Local/WSL .env file
    load_dotenv(override=True)
    
    # 3. Fallback to standard environment variable (os.environ)
    return os.getenv(key_name)

# Usage
HF_TOKEN = get_secret('HF_TOKEN')
WANDB_API_KEY = get_secret('WANDB_API_KEY')

# Apply to environment so libraries find them automatically
if HF_TOKEN:
    # HUGGINGFACE env
    os.environ['HF_TOKEN'] = HF_TOKEN
    from huggingface_hub import login
    login(token=HF_TOKEN)
    print("login to hf")
else:
    print("login to hf failed")

if WANDB_API_KEY:
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY    
    # WANDB env
    import wandb
    wandb.login()
    print("login to wandb")
else:
    print("login to wandb failed")


In [ ]:
# Prepare training data set. Combine multiple batches under one training data.
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import login
from google.colab import userdata

# Ensure Hugging Face login is done, if not already
# login(token=userdata.get('HF_TOKEN')) # Uncomment if not logged in already

# Generate input_datasets using a loop
input_datasets = [f'fanaustinca/fll_diffusion_v2_batch_{i:02d}' for i in range(1, 4)]

output_dataset_name = 'fanaustinca/fll_training_150episodes'

# Load datasets
print("Loading datasets...")
datasets_list = []
for dataset_name in input_datasets:
    print(f"  Loading {dataset_name}")
    dataset = load_dataset(dataset_name, trust_remote_code=True)
    # Assuming the dataset is a DatasetDict and we want to combine the 'train' split
    # Adjust if your dataset structure is different (e.g., just a Dataset object)
    if 'train' in dataset:
        datasets_list.append(dataset['train'])
    else:
        datasets_list.append(dataset)

# Concatenate datasets
print("Concatenating datasets...")
combined_dataset = concatenate_datasets(datasets_list)

# Push to Hugging Face Hub
print(f"Pushing combined dataset to {output_dataset_name}...")
combined_dataset.push_to_hub(output_dataset_name, private=False)
print("Dataset combined and pushed successfully!")

In [ ]:
# Adjust the following variables before starting training
tag = '150episodes'
hf_namespace = 'fanaustinca'
training_repo = f"{hf_namespace}/fll_combined_150episodes"
model_repo = f"{hf_namespace}/fll_act_model_150episodes"
# Use 'local_output' consistently
local_output = f"outputs/model/{model_repo}"
steps = 150000
batch_size = 32
num_workers = 2

# Clean up output from last run
!rm -rf {local_output}

"""
tfs_config = [
    dict(type="ToImage"),
    dict(type="RandomAffine", degrees=0, translate=(0.05, 0.05), fill=0),
    dict(type="ColorJitter", brightness=.2, contrast=.2, saturation=.2, hue=.05),
    dict(type="ConvertImageDtype", dtype="float32"),
]
"""

tfs_config = {
  "to_tensor": {"type": "ToImage"},
  "affine": {
    "type": "RandomAffine",
    "kwargs": {"degrees": 0, "translate": [0.05, 0.05]}
  },
  "jitter": {
    "type": "ColorJitter",
    "kwargs": {"brightness": 0.1, "contrast": 0.1, "saturation": 0.1, "hue": 0.05}
  }
}

# Convert to a compact JSON string for the CLI
import json
tfs_string = json.dumps(tfs_config, separators=(',', ':'))

!python src/lerobot/scripts/lerobot_train.py \
    --policy.type=act \
    --dataset.repo_id={training_repo} \
    --job_name=flamebots_act_training \
    --output_dir={local_output} \
    --policy.n_action_steps=100 \
    --policy.device=cuda \
    --steps={steps} \
    --save_freq=5000 \
    --batch_size={batch_size} \
    --log_freq=1000 \
    --num_workers={num_workers} \
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=true \
    --dataset.image_transforms.tfs='{tfs_string}' \
    --policy.use_amp=true \
    --policy.push_to_hub=true \
    --policy.repo_id={model_repo} \
    --wandb.enable=true

#!hf repo tag create {model_repo} {tag} -m "Diffusion model trained on {training_repo}, tag {tag}"